## Import libraries

- Follow instructions on this link to sign up for Google Earth Engine - https://geemap.org/installation/

In [1]:
import ee

# ee.Authenticate()
ee.Initialize()

In [2]:
import sys
import geemap

# import geemap.colormaps as cm

In [3]:
print(
    f"""System: {sys.version} 
geemap: {geemap.__version__}
ee: {ee.__version__} 
        """
)

System: 3.12.7 | packaged by conda-forge | (main, Oct  4 2024, 15:57:01) [Clang 17.0.6 ] 
geemap: 0.34.5
ee: 1.1.3 
        


In [ ]:
# example AOI to search Landsat images
polygon = ee.Geometry.Polygon(
    [
        [
            [-92.625046, 38.351888],
            [-92.625046, 38.416342],
            [-92.52943, 38.416342],
            [-92.52943, 38.351888],
            [-92.625046, 38.351888],
        ]
    ]
)

## Get Sentinel-2 Image collection

In [5]:
# SCL Mask
def sclmask(image):
    """Function to mask clouds, shadows, and snow using the Sentinel-2 SCL band
    Args: {ee.Image} Sentinel-2 image
    Returns:  {ee.Image} cloud, shadow, and snow masked Sentinel-2 image"""

    scl = image.select("SCL")

    # Define the mask conditions
    wantedPixels = scl.gt(3).And(scl.lt(8))
    return (
        image.updateMask(wantedPixels)
        .divide(10000)
        .copyProperties(image, ["system:time_start"])
    )

In [6]:
year = 2019
month = 5
day = 1

# Define the start and end dates for the entire year.
start_date = ee.Date.fromYMD(year, month, day)
end_date = start_date.advance(5, "month")

print(start_date.format().getInfo())
print(f"{end_date.format().getInfo()}")

2019-05-01T00:00:00
2019-10-01T00:00:00


In [7]:
s2_coll = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterDate(start_date, end_date)
    .filterBounds(polygon)
    .filter(ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 30))
    .map(lambda img: img.clip(polygon)))

In [8]:
# get Sentinel-2 image collection
s2_coll = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterDate(start_date, end_date)
    .filterBounds(polygon)
    .filter(ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 30))
    .map(lambda img: img.clip(polygon))
    .map(sclmask)
    .select(["B2", "B3", "B4", "B8"])
    .sort("system:time_start")
)
print(f"Number of images: {s2_coll.size().getInfo()}")

Number of images: 20


## Visualize using GeeMap

In [9]:
# Visualize using geemap
Map = geemap.Map(center=(40, -100), zoom=4, basemap="HYBRID")
Map.addLayer(polygon, {}, "AOI")
Map.centerObject(polygon)
visualization = {
    "bands": ["B4_median", "B3_median", "B2_median"],
    "min": 0.0,
    "max": 0.3,
}

Map.addLayer(
    s2_coll.reduce(ee.Reducer.median()), visualization, "Median RGB"
)

Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…